# Collaborative Filtering

**Item Based**: which takes similarities between items’ consumption histories

**User Based**: that considers similarities between user consumption histories and item similarities

In [1]:
#Import libraries
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
data = pd.read_csv("data/groceries.csv")

In [3]:
data.head(100)

,Person,item
0,1,citrus fruit
1,1,semi-finished bread
2,1,margarine
3,1,ready soups
4,2,tropical fruit
...,...,...
95,31,specialty bar
96,32,hamburger meat
97,32,other vegetables
98,32,rolls/buns


In [4]:
#Assume that for all items only one quantity was bought 

**Exercise 1** Add a column to data : `Quantity` that has value 1 

In [5]:
data["Quantity"] = 1

In [6]:
data.head()

,Person,item,Quantity
0,1,citrus fruit,1
1,1,semi-finished bread,1
2,1,margarine,1
3,1,ready soups,1
4,2,tropical fruit,1


In [7]:
len(pd.unique(data.item))

169

In [8]:
#This particular view isn't very helpful for us for analysis.
#This way of data being arranged is called LONG
#We need it in wide format

In [9]:
#Converting data from long to wide format
dataWide = data.pivot("Person", "item", "Quantity")

In [10]:
dataWide.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Person,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


**Exercise 2**
Print the data for Person number 2

In [11]:
dataWide[dataWide.index==2]

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Person,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [12]:
dataWide.iloc[1:2,:]

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Person,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [13]:
dataWide.loc[2,:]

item
Instant food products    NaN
UHT-milk                 NaN
abrasive cleaner         NaN
artif. sweetener         NaN
baby cosmetics           NaN
                        ... 
white bread              NaN
white wine               NaN
whole milk               NaN
yogurt                   1.0
zwieback                 NaN
Name: 2, Length: 169, dtype: float64

**Exercise 3** Print the data for row number 2

In [14]:
dataWide.iloc[1,:]

item
Instant food products    NaN
UHT-milk                 NaN
abrasive cleaner         NaN
artif. sweetener         NaN
baby cosmetics           NaN
                        ... 
white bread              NaN
white wine               NaN
whole milk               NaN
yogurt                   1.0
zwieback                 NaN
Name: 2, Length: 169, dtype: float64

In [15]:
#Replace NA with 0 
dataWide.fillna(0, inplace=True)

In [16]:
dataWide.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Person,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Item-based Collaborative Filtering

In item based collaborative filtering we do not care about the user column

In [17]:
#Drop the Person column
data_ib = dataWide.copy()

In [18]:
data_ib.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Person,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
data_ib = data_ib.reset_index()

In [20]:
data_ib.head()


item,Person,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
#Drop the Person column
#data_ib = data_ib.iloc[:,1:]
data_ib = data_ib.drop("Person", axis=1)

In [22]:
data_ib.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [23]:
# Create a placeholder dataframe listing item vs. item
data_ibs = pd.DataFrame(index=data_ib.columns,
                        columns=data_ib.columns)

In [24]:
data_ibs.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
item,,,,,,,,,,,,,,,,,,,,,
Instant food products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UHT-milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abrasive cleaner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
artif. sweetener,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
baby cosmetics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Similarity Measure 

We will now find similarities.

We will use `cosine similarity`

<img src="img/cosine.png" >

The resulting similarity ranges from −1 meaning exactly opposite, to 1 meaning exactly the same, with 0 indicating orthogonality (decorrelation), and in-between values indicating intermediate similarity or dissimilarity.

*src* https://en.wikipedia.org/wiki/Cosine_similarity

In essense the cosine similarity takes the sum product of the first and second column, then divides that by the product of the square root of the sum of squares of each column.

In [26]:
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(data_ib.iloc[:,i],data_ib.iloc[:,j])

In [27]:
data_ibs.head()

item,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
item,,,,,,,,,,,,,,,,,,,,,
Instant food products,1,0.024811,0.0,0.0,0.0,0.0,0.0,0.025588,0.0,0.039623,...,0.025158,0.014064,0.01736,0.029661,0.0,0.044236,0.0,0.06733,0.042524,0.040931
UHT-milk,0.024811,1,0.0,0.038984,0.0,0.0,0.0,0.037616,0.02122,0.033979,...,0.03082,0.027566,0.059549,0.062291,0.0,0.035225,0.012095,0.042891,0.108655,0.040114
abrasive cleaner,0.0,0.0,1,0.029881,0.0,0.0,0.0,0.0,0.03253,0.044647,...,0.0,0.042258,0.017388,0.050929,0.0,0.024922,0.012361,0.05395,0.045634,0.0
artif. sweetener,0.0,0.038984,0.029881,1,0.0,0.0,0.0,0.040204,0.0,0.007782,...,0.0,0.022097,0.009092,0.006658,0.0,0.017376,0.0,0.03879,0.052498,0.0
baby cosmetics,0.0,0.0,0.0,0.0,1,0.0,0.0,0.030949,0.0,0.0,...,0.0,0.0,0.020998,0.015376,0.0,0.020064,0.0,0.024431,0.0,0.0


With our similarity matrix filled out we can look for each items “neighbour” by looping through ‘data_ibs’, sorting each column in descending order, and grabbing the name of each of the top 3 products.

In [29]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,4))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:3] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:3].index

In [30]:
data_neighbours

,1,2,3
item,,,
Instant food products,Instant food products,hamburger meat,canned fish
UHT-milk,UHT-milk,bottled water,yogurt
abrasive cleaner,abrasive cleaner,preservation products,cleaner
artif. sweetener,artif. sweetener,potato products,salad dressing
baby cosmetics,baby cosmetics,cream,dish cleaner
...,...,...,...
white bread,white bread,whole milk,processed cheese
white wine,white wine,bottled water,shopping bags
whole milk,whole milk,other vegetables,yogurt


**Exercise 4** Modify the above code to print the top 10 similar products for each product

In [32]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:10].index
    
data_neighbours

,1,2,3,4,5,6,7,8,9,10
item,,,,,,,,,,
Instant food products,Instant food products,hamburger meat,canned fish,other vegetables,whole milk,root vegetables,curd,rolls/buns,butter,kitchen utensil
UHT-milk,UHT-milk,bottled water,yogurt,other vegetables,soda,coffee,margarine,domestic eggs,brown bread,citrus fruit
abrasive cleaner,abrasive cleaner,preservation products,cleaner,curd cheese,root vegetables,dish cleaner,other vegetables,salad dressing,rice,berries
artif. sweetener,artif. sweetener,potato products,salad dressing,skin care,candles,flour,rum,yogurt,frankfurter,mustard
baby cosmetics,baby cosmetics,cream,dish cleaner,cookware,syrup,kitchen towels,soups,butter milk,oil,sweet spreads
...,...,...,...,...,...,...,...,...,...,...
white bread,white bread,whole milk,processed cheese,ham,other vegetables,fruit/vegetable juice,tropical fruit,soda,yogurt,pip fruit
white wine,white wine,bottled water,shopping bags,soda,sausage,bottled beer,prosecco,canned beer,rolls/buns,fruit/vegetable juice
whole milk,whole milk,other vegetables,yogurt,root vegetables,rolls/buns,tropical fruit,whipped/sour cream,domestic eggs,butter,curd


# User Based collaborative Filtering

The process for creating a User Based recommendation system is as follows:

1. Have Item-Based similarity matrix
2. Check which items the user has consumed
3. For each item the user has consumed, get the top X neighbours
4. Get the consumption record of the user for each neighbour.
5. Compute similarity score
6. Recommend the items with the highest score

In [33]:
#Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

#Understand what this function does ! 

In [34]:
data_sims1 = dataWide.reset_index()

In [35]:
data_sims1.head()

item,Person,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [36]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data_sims1.index,columns=data_sims1.columns)
data_sims.iloc[:,:1] = data_sims1.iloc[:,:1]

In [37]:
#This is the same as our original data but with nothing filled in except the headers
data_sims.head()

item,Person,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
data_sims12 = data_sims1.iloc[:500,:]

In [39]:
data_sims11 = data_sims.iloc[:500,:]

In [43]:
#Need to run this for only 500 users. Might be slow beyond that. 
#Subset it to 500 users before running this
for i in range(0,len(data_sims11.index)):
    for j in range(1,len(data_sims11.columns)):
        user = data_sims11.index[i]
        product = data_sims11.columns[j]
 
        if data_sims12.loc[i][j] == 1:
            data_sims11.loc[i][j] = 0
        else:
            product_top_names = data_neighbours.loc[product][1:10]
            product_top_sims = data_ibs.loc[product].sort_values(ascending=False)[1:10]
            user_purchases = data_ib.loc[user,product_top_names]
            
            print (i)
            print (j)
 
            data_sims11.iloc[i][j] = getScore(user_purchases,product_top_sims)

0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
0
15
0
16
0
17
0
18
0
19
0
20
0
21
0
22
0
23
0
24
0
25
0
26
0
27
0
28
0
29
0
30
0
31
0
33
0
34
0
35
0
36
0
37
0
38
0
39
0
40
0
41
0
42
0
43
0
44
0
45
0
46
0
47
0
48
0
49
0
50
0
51
0
52
0
53
0
54
0
55
0
56
0
57
0
58
0
59
0
60
0
61
0
62
0
63
0
64
0
65
0
66
0
67
0
68
0
69
0
70
0
71
0
72
0
73
0
74
0
75
0
76
0
77
0
78
0
79
0
80
0
81
0
82
0
83
0
84
0
85
0
86
0
87
0
88
0
89
0
91
0
92
0
93
0
94
0
95
0
96
0
97
0
98
0
99
0
100
0
101
0
102
0
103
0
104
0
105
0
106
0
107
0
108
0
109
0
110
0
111
0
112
0
113
0
114
0
115
0
116
0
117
0
118
0
119
0
121
0
122
0
123
0
124
0
125
0
126
0
127
0
128
0
129
0
130
0
131
0
132
0
133
0
135
0
136
0
137
0
138
0
139
0
140
0
141
0
142
0
143
0
144
0
145
0
146
0
147
0
148
0
149
0
150
0
151
0
152
0
153
0
154
0
155
0
156
0
157
0
158
0
159
0
160
0
161
0
162
0
163
0
164
0
165
0
166
0
167
0
168
0
169
1
1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
1
22
1
23
1
24
1
25


9
114
9
115
9
116
9
117
9
118
9
119
9
120
9
121
9
122
9
123
9
124
9
125
9
126
9
127
9
128
9
129
9
130
9
131
9
132
9
133
9
134
9
135
9
136
9
137
9
138
9
139
9
140
9
141
9
142
9
143
9
144
9
145
9
146
9
147
9
148
9
149
9
150
9
151
9
152
9
153
9
154
9
155
9
156
9
157
9
158
9
159
9
160
9
161
9
162
9
163
9
164
9
165
9
166
9
168
9
169
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
8
10
9
10
10
10
11
10
12
10
13
10
15
10
16
10
17
10
18
10
19
10
20
10
21
10
22
10
23
10
24
10
25
10
26
10
27
10
28
10
29
10
31
10
32
10
33
10
34
10
35
10
36
10
37
10
38
10
39
10
40
10
41
10
42
10
43
10
44
10
45
10
46
10
47
10
48
10
49
10
50
10
51
10
52
10
53
10
54
10
55
10
56
10
57
10
58
10
59
10
60
10
61
10
62
10
63
10
64
10
65
10
66
10
67
10
68
10
69
10
70
10
71
10
72
10
73
10
74
10
75
10
76
10
77
10
78
10
79
10
80
10
81
10
82
10
83
10
84
10
85
10
86
10
87
10
88
10
89
10
90
10
91
10
92
10
93
10
94
10
95
10
96
10
97
10
98
10
99
10
100
10
101
10
102
10
103
10
105
10
106
10
107
10
108
10
109
10
110
10
111
10
112
10
113
10
114

18
40
18
41
18
42
18
43
18
44
18
45
18
46
18
47
18
48
18
49
18
50
18
51
18
52
18
53
18
54
18
55
18
56
18
57
18
58
18
59
18
60
18
61
18
62
18
63
18
64
18
65
18
66
18
67
18
68
18
69
18
70
18
71
18
72
18
73
18
74
18
75
18
76
18
77
18
78
18
79
18
80
18
81
18
82
18
83
18
84
18
85
18
86
18
87
18
88
18
89
18
90
18
91
18
92
18
93
18
94
18
95
18
96
18
97
18
98
18
99
18
100
18
101
18
102
18
103
18
104
18
105
18
106
18
107
18
108
18
109
18
110
18
111
18
112
18
113
18
114
18
115
18
116
18
117
18
118
18
119
18
120
18
121
18
122
18
123
18
124
18
125
18
126
18
127
18
128
18
129
18
130
18
131
18
132
18
133
18
134
18
135
18
136
18
137
18
138
18
139
18
140
18
141
18
142
18
143
18
144
18
145
18
146
18
147
18
148
18
149
18
150
18
151
18
152
18
153
18
154
18
155
18
156
18
157
18
158
18
159
18
160
18
161
18
162
18
163
18
164
18
165
18
166
18
167
18
168
18
169
19
1
19
2
19
3
19
4
19
5
19
6
19
7
19
8
19
9
19
10
19
11
19
12
19
13
19
14
19
15
19
16
19
17
19
18
19
19
19
20
19
21
19
22
19
23
19
24
19
25
19
26
19


26
45
26
46
26
47
26
48
26
49
26
50
26
51
26
52
26
53
26
54
26
55
26
56
26
57
26
58
26
59
26
60
26
61
26
62
26
63
26
64
26
65
26
66
26
67
26
68
26
69
26
70
26
71
26
72
26
73
26
74
26
75
26
76
26
77
26
78
26
79
26
80
26
81
26
82
26
83
26
84
26
85
26
86
26
87
26
88
26
89
26
90
26
91
26
92
26
93
26
94
26
95
26
96
26
97
26
98
26
99
26
100
26
101
26
102
26
103
26
104
26
105
26
106
26
107
26
108
26
109
26
110
26
111
26
112
26
113
26
114
26
115
26
116
26
117
26
118
26
119
26
120
26
121
26
122
26
123
26
124
26
125
26
126
26
127
26
128
26
129
26
130
26
131
26
132
26
133
26
134
26
135
26
136
26
137
26
138
26
139
26
140
26
141
26
142
26
143
26
144
26
145
26
146
26
147
26
148
26
149
26
150
26
151
26
152
26
153
26
154
26
155
26
156
26
157
26
158
26
159
26
160
26
161
26
162
26
163
26
164
26
165
26
166
26
167
26
169
27
1
27
2
27
3
27
4
27
5
27
6
27
7
27
8
27
9
27
10
27
11
27
12
27
13
27
14
27
15
27
16
27
17
27
18
27
19
27
20
27
21
27
22
27
23
27
24
27
25
27
26
27
27
27
28
27
29
27
31
27
32
27
33
27
3

34
27
34
28
34
29
34
30
34
31
34
32
34
33
34
34
34
35
34
36
34
37
34
38
34
39
34
40
34
41
34
42
34
43
34
44
34
45
34
46
34
48
34
49
34
50
34
51
34
52
34
53
34
54
34
55
34
56
34
57
34
58
34
59
34
60
34
61
34
62
34
63
34
64
34
65
34
66
34
68
34
69
34
70
34
71
34
72
34
73
34
74
34
75
34
76
34
77
34
78
34
79
34
80
34
81
34
82
34
83
34
84
34
85
34
86
34
87
34
88
34
89
34
90
34
91
34
92
34
93
34
94
34
95
34
96
34
97
34
98
34
99
34
100
34
101
34
102
34
103
34
104
34
105
34
106
34
107
34
108
34
109
34
110
34
111
34
112
34
113
34
114
34
115
34
116
34
117
34
118
34
119
34
120
34
121
34
122
34
123
34
124
34
125
34
126
34
127
34
128
34
129
34
130
34
131
34
132
34
133
34
134
34
135
34
136
34
137
34
138
34
139
34
140
34
141
34
142
34
143
34
144
34
145
34
146
34
147
34
148
34
149
34
150
34
151
34
152
34
153
34
154
34
155
34
156
34
157
34
158
34
159
34
160
34
161
34
162
34
163
34
164
34
165
34
166
34
167
34
168
34
169
35
1
35
2
35
3
35
4
35
5
35
6
35
7
35
8
35
9
35
10
35
11
35
12
35
13
35
14
35
15
35


...

465
17
465
18
465
19
465
20
465
21
465
22
465
23
465
24
465
25
465
26
465
27
465
28
465
29
465
30
465
31
465
32
465
33
465
34
465
35
465
36
465
37
465
38
465
39
465
40
465
41
465
43
465
44
465
45
465
46
465
47
465
48
465
49
465
50
465
51
465
52
465
53
465
54
465
55
465
56
465
57
465
59
465
60
465
61
465
62
465
63
465
64
465
66
465
67
465
68
465
69
465
70
465
71
465
72
465
73
465
74
465
76
465
77
465
78
465
79
465
80
465
81
465
82
465
83
465
84
465
85
465
87
465
88
465
89
465
90
465
91
465
92
465
93
465
94
465
95
465
96
465
97
465
98
465
99
465
100
465
101
465
102
465
103
465
104
465
105
465
106
465
107
465
108
465
110
465
111
465
112
465
113
465
114
465
115
465
116
465
117
465
118
465
119
465
120
465
121
465
122
465
123
465
126
465
127
465
128
465
129
465
130
465
131
465
133
465
135
465
136
465
137
465
138
465
139
465
140
465
141
465
142
465
143
465
144
465
145
465
146
465
147
465
148
465
149
465
150
465
151
465
152
465
153
465
154
465
155
465
156
465
157
465
158
465
159
465
160
465
16

471
157
471
158
471
159
471
160
471
161
471
162
471
163
471
164
471
165
471
166
471
167
471
168
471
169
472
1
472
2
472
3
472
4
472
5
472
6
472
7
472
8
472
9
472
10
472
11
472
12
472
13
472
14
472
15
472
16
472
18
472
19
472
20
472
21
472
22
472
23
472
24
472
25
472
26
472
27
472
28
472
29
472
30
472
31
472
32
472
33
472
34
472
35
472
36
472
37
472
38
472
39
472
40
472
41
472
42
472
43
472
44
472
45
472
46
472
47
472
48
472
49
472
50
472
51
472
52
472
53
472
54
472
55
472
56
472
57
472
58
472
59
472
60
472
61
472
62
472
63
472
64
472
65
472
66
472
67
472
68
472
69
472
70
472
71
472
72
472
73
472
74
472
76
472
77
472
78
472
79
472
80
472
81
472
82
472
83
472
84
472
85
472
86
472
87
472
88
472
89
472
90
472
91
472
92
472
93
472
94
472
95
472
96
472
97
472
98
472
99
472
100
472
102
472
103
472
104
472
105
472
106
472
108
472
109
472
110
472
111
472
112
472
114
472
115
472
116
472
117
472
118
472
119
472
120
472
121
472
122
472
123
472
125
472
126
472
128
472
129
472
130
472
131
472
132
47

479
49
479
50
479
51
479
52
479
53
479
54
479
55
479
56
479
57
479
58
479
59
479
60
479
61
479
62
479
63
479
64
479
65
479
66
479
67
479
68
479
69
479
70
479
71
479
72
479
73
479
74
479
75
479
77
479
78
479
79
479
80
479
81
479
82
479
83
479
84
479
85
479
86
479
87
479
88
479
89
479
90
479
91
479
92
479
93
479
94
479
95
479
96
479
97
479
98
479
99
479
100
479
101
479
102
479
103
479
104
479
105
479
106
479
107
479
108
479
109
479
110
479
111
479
112
479
113
479
114
479
115
479
116
479
117
479
118
479
119
479
120
479
121
479
122
479
123
479
124
479
125
479
126
479
127
479
128
479
129
479
130
479
131
479
132
479
133
479
134
479
135
479
136
479
137
479
138
479
139
479
140
479
141
479
142
479
143
479
144
479
145
479
146
479
147
479
148
479
149
479
150
479
151
479
152
479
153
479
154
479
155
479
156
479
157
479
158
479
159
479
160
479
161
479
162
479
163
479
164
479
165
479
166
479
167
479
168
479
169
480
1
480
2
480
3
480
4
480
5
480
6
480
7
480
8
480
9
480
10
480
11
480
12
480
13
480
15
4

486
97
486
98
486
99
486
100
486
101
486
102
486
103
486
105
486
106
486
107
486
108
486
109
486
110
486
111
486
112
486
113
486
114
486
115
486
116
486
117
486
118
486
119
486
120
486
121
486
122
486
123
486
124
486
125
486
126
486
127
486
128
486
129
486
130
486
131
486
132
486
133
486
134
486
135
486
136
486
137
486
138
486
139
486
141
486
142
486
143
486
144
486
145
486
146
486
147
486
148
486
149
486
150
486
151
486
152
486
153
486
154
486
155
486
156
486
157
486
158
486
159
486
160
486
161
486
162
486
163
486
164
486
165
486
166
486
167
486
168
486
169
487
1
487
2
487
3
487
4
487
5
487
6
487
7
487
8
487
9
487
10
487
11
487
12
487
13
487
14
487
15
487
16
487
17
487
18
487
19
487
20
487
22
487
23
487
24
487
25
487
26
487
27
487
28
487
29
487
30
487
31
487
32
487
33
487
34
487
35
487
36
487
37
487
38
487
39
487
40
487
41
487
42
487
43
487
44
487
45
487
47
487
48
487
49
487
50
487
51
487
52
487
53
487
54
487
55
487
56
487
57
487
58
487
59
487
60
487
61
487
62
487
63
487
64
487
65
487

493
125
493
126
493
127
493
128
493
129
493
130
493
131
493
132
493
133
493
134
493
135
493
136
493
137
493
138
493
139
493
140
493
141
493
142
493
143
493
144
493
145
493
146
493
147
493
148
493
149
493
150
493
151
493
152
493
153
493
154
493
155
493
156
493
157
493
158
493
159
493
160
493
161
493
162
493
163
493
164
493
165
493
166
493
168
493
169
494
1
494
2
494
3
494
4
494
5
494
6
494
7
494
8
494
9
494
10
494
11
494
12
494
13
494
14
494
15
494
16
494
17
494
18
494
19
494
20
494
21
494
22
494
23
494
24
494
25
494
26
494
27
494
28
494
29
494
30
494
31
494
32
494
33
494
34
494
35
494
36
494
37
494
38
494
39
494
40
494
41
494
42
494
43
494
44
494
45
494
46
494
47
494
48
494
49
494
50
494
51
494
52
494
53
494
54
494
55
494
56
494
57
494
58
494
59
494
60
494
61
494
62
494
63
494
64
494
65
494
66
494
67
494
68
494
69
494
70
494
71
494
72
494
73
494
74
494
75
494
76
494
77
494
78
494
79
494
80
494
81
494
82
494
83
494
84
494
85
494
86
494
87
494
88
494
89
494
90
494
91
494
92
494
93
494
94

In [44]:
# Get the top products
data_recommend = pd.DataFrame(index=data_sims.index, columns=['Person','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]


In [45]:
# Instead of top product scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

In [47]:
# Print a sample
print( data_recommend.iloc[:10,:4])

  Person                  1               2                  3
0      1            candles      hair spray                oil
1      2  seasonal products       pip fruit          beverages
2      3            cereals            curd      domestic eggs
3      4        butter milk  tropical fruit        canned fish
4      5             coffee         cereals          chocolate
5      6               curd         cleaner      domestic eggs
6      7        frankfurter         sausage      spread cheese
7      8     red/blush wine          dishes      bottled water
8      9           prosecco        dog food  finished products
9     10               soap            curd      domestic eggs
